In [42]:
# 모멘텀 투자 예시 : 최근 6~12 개월간 최강세주 상위 10~30%를 매수해 1~3개월 보유한 후 재평가해서 포트폴리오를 리밸런싱하기
# 기간별 모멘텀에 대한 여루 연구 결과를 종합하면 3~12개월 동안의 중기 매매에서는 추세 지속 현상이 발생하지만, 1개월 이내의 단기 매매나 2년 이상의 장기 매매에서는 상승 또는 하락했던 주가가 제 위치를 찾아가는 역추세 현상이 나타나는 것으로 알려져 있음.

# 위의 최근 6~12개월 동안 상대적으로 수익률이 높은 종목을 매수하는 상대적 모멘텀 전략은 나름대로 일리가 있어 보이지만, 반면에 이미 수익이 난 종목을 매수하기 때문에 소위 상투를 잡게 될 위험성이 커짐. 
# 이에 대한 해결책이 바로 절대적 모멘텀 전략으로 상승장에서만 투자하고 하락장에서는 미국 단기 국채나 현금으로 갈아타는 전략.
# 즉, 투자하는 종목들의 최근 수익률이 채권 수익률에 비해 높을 때 매수하고, 채권 수익률에 미치지 못하면 해당 종목들을 매도하고 단기 국채를 매수하거나 현금을 보유하는 방법.

# 게리 안토나치의 듀얼 모멘텀 투자(Dual Momentum Investing)은 상대 강도가 센 주식 종목들에 투자하는 상대적 모멘텀 전략과, 과거 6~12개월의 수익이 단기 국채 수익률을 능가하는 강세장에서만 투자하는 절대적 모멘텀 전략을 하나로 합친 듀얼 전략임

# 절대 모멘텀 자체는 상승장에서 투자하고 하락장에서 쉬어가는 매우 단순한 전략이지만, 상대 모멘텀과 함께 사용함으로써 상대 모멘텀만 사용했을 때보다 MDD를 줄일 뿐만 아니라 더 높은 수익률을 달성할 수 있음

In [1]:
import pandas as pd
import sqlite3
import datetime

# 날짜 체크 해보기
# 삼성전자
code = "005930"
check_start_date='2015-01-01'
# SQLite3 DB 불러오기
con = sqlite3.connect("C:\\Users\\inventor\\Desktop\\FR study\\종목가격정보.db")

# 상대 모멘텀을 구할 시작 일자
sql = f"SELECT max(date) FROM '{code}' where date<='{check_start_date}'"
pointer = con.execute(sql)
result = pointer.fetchone()
print(result)
start_date = datetime.datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')
start_date = start_date.strftime("%Y-%m-%d")
print(start_date)

check_end_date='2020-01-01'
# 상대 모멘텀을 구할 종료 일자
sql = f"SELECT max(date) FROM '{code}' where date<='{check_end_date}'"
pointer = con.execute(sql)
result = pointer.fetchone()
print(result)
end_date = datetime.datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')
end_date = end_date.strftime("%Y-%m-%d")
print(end_date)
print(type(end_date))
con.close()

('2014-12-30 00:00:00',)
2014-12-30
('2019-12-30 00:00:00',)
2019-12-30
<class 'str'>


In [57]:
# 종목코드 관련 SQLite3 DB 불러오기
con = sqlite3.connect('C:\\Users\\inventor\\Desktop\\FR study\\상장법인목록.db')
code_data = pd.read_sql("SELECT * FROM CorpList", con)
con.close()

code_data = code_data[['종목코드', '기업명']]
# print(code_data['종목코드'])

In [64]:
# 특정 기간 동안 수익률이 제일 높았던 stock_count개의 종목들 (상대 모멘텀)
def get_rltv_momentum(start_date, end_date, stock_count):
    
    rows=[]
    columns=['code', 'company', 'old_price', 'new_price', 'returns']

    # SQLite3 DB 불러오기
    con = sqlite3.connect("C:\\Users\\inventor\\Desktop\\FR study\\종목가격정보.db")
    
    # 임시 체크 코드 
    code = code_data['종목코드'][0]
    # 상대 모멘텀을 구할 시작 일자
    sql = f"SELECT max(date) FROM '{code}' where date<='{start_date}'"
    pointer = con.execute(sql)
    result = pointer.fetchone()
    # print(result)
    start_date = datetime.datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')

    # 상대 모멘텀을 구할 종료 일자
    sql = f"SELECT max(date) FROM '{code}' where date<='{end_date}'"
    pointer = con.execute(sql)
    result = pointer.fetchone()
    # print(result)
    end_date = datetime.datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')

    # KRX 종목별 수익률을 구해서 2차원 리스트 형태로 추가
    for code, company in zip(code_data['종목코드'], code_data['기업명']):
        # print(type(code))
        sql = f"SELECT Close FROM '{code}' where date='{start_date}'"
        pointer = con.execute(sql)
        result = pointer.fetchone()

        if (result is None):
            continue
        
        old_price = int(result[0])
        # print(old_price)

        sql = f"SELECT Close FROM '{code}' where date='{end_date}'"
        pointer = con.execute(sql)
        result = pointer.fetchone()

        if (result is None):
            continue
            
        new_price = int(result[0])
        # print(new_price)

        returns = (new_price / old_price - 1) * 100 
        rows.append([code, company, old_price, new_price, returns])
    
    # 상대 모멘텀 데이터프레임을 생성한 후 수익률순으로 출력
    df = pd.DataFrame(rows, columns=columns)
    df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
    df = df.sort_values(by='returns', ascending=False)
    df = df.head(stock_count)
    df.index = pd.Index(range(stock_count))
    con.close()
    print(df)
    print(f"\nRelative momentum ({start_date} ~ {end_date}) : {df['returns'].mean():.2f}% \n")
    
    return df

In [68]:
# 6개월 동안 300 종목의 상대 모멘텀 구하기
df_relative = get_rltv_momentum('2018-07-01', '2018-12-31', 300)

code company  old_price  new_price     returns
0    258250     셀젠텍       6100      18000  195.081967
1    224760  엔에스컴퍼니      19000      55000  189.473684
2    009460    한창제지       1030       2970  188.349515
3    143160    아이디스       7400      21150  185.810811
4    051630    진양화학       2240       6300  181.250000
..      ...     ...        ...        ...         ...
295  112190    KC산업       2400       2500    4.166667
296  023150   MH에탄올       5660       5890    4.063604
297  001720    신영증권      56700      59000    4.056437
298  051910    LG화학     333500     347000    4.047976
299  033660   아주캐피탈       8200       8530    4.024390

[300 rows x 5 columns]

Relative momentum (2018-06-29 00:00:00 ~ 2018-12-28 00:00:00) : 27.90% 



In [74]:
# 특정 기간 동안 상대 모멘텀에 투자했을 때의 평균 수익률 (절대 모멘텀)
# 자산의 가치가 상승하고 있을 때만 투자하고 그렇지 않을 때는 단기 국채를 매수하거나 현금을 보유하는 전략
# 이 함수는 상대 모멘텀의 수익률이 국채 수익률을 상회했다고 가정하고 따라서 절대 모멘텀 룰에 따라 시장에 참여하는 코드임
def get_abs_momentum(rltv_momentum, start_date, end_date):

    stock_list=list(rltv_momentum['code'])

    # SQLite3 DB 불러오기
    con = sqlite3.connect("C:\\Users\\inventor\\Desktop\\FR study\\종목가격정보.db")

    # 임시 체크 코드 
    code = stock_list[0]
    # 상대 모멘텀을 구할 시작 일자
    sql = f"SELECT max(date) FROM '{code}' where date<='{start_date}'"
    pointer = con.execute(sql)
    result = pointer.fetchone()
    # print(result)
    start_date = datetime.datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')

    # 상대 모멘텀을 구할 종료 일자
    sql = f"SELECT max(date) FROM '{code}' where date<='{end_date}'"
    pointer = con.execute(sql)
    result = pointer.fetchone()
    # print(result)
    end_date = datetime.datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')

    # 상대 모멘텀의 종목별 수익률을 구해서 2차원 리스트 형태로 추가
    rows = []
    columns = ['code', 'company', 'old_price', 'new_price', 'returns']
    for _, code in enumerate(stock_list):
        # print(type(code))
        sql = f"SELECT Close FROM '{code}' where date='{start_date}'"
        pointer = con.execute(sql)
        result = pointer.fetchone()

        if (result is None):
            continue
        
        old_price = int(result[0])
        # print(old_price)

        sql = f"SELECT Close FROM '{code}' where date='{end_date}'"
        pointer = con.execute(sql)
        result = pointer.fetchone()

        if (result is None):
            continue
            
        new_price = int(result[0])
        # print(new_price)

        returns = (new_price / old_price - 1) * 100
        company = code_data.query("종목코드=='{}'".format(code))['기업명'].to_string(index=False)
        rows.append([code, company, old_price, new_price, returns])
    
    # 절대 모멘텀 데이터프레임을 생성한 후 수익률순으로 출력
    df = pd.DataFrame(rows, columns=columns)
    df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
    df = df.sort_values(by='returns', ascending=False)

    con.close()
    print(df)
    print(f"\nAbsolute momentum ({start_date} ~ {end_date}) : {df['returns'].mean():.2f}% \n")

In [75]:
# 앞에서 구한 6개월 동안 300 종목의 상대 모멘텀 날짜 2018-07-01 ~ 2018-12-31 이후 강세주 300종목의 6개월 수익률
df_absolute = get_abs_momentum(df_relative,'2019-01-01', '2019-06-30')

code   company  old_price  new_price     returns
45   078070   유비쿼스홀딩스       7980      47100  490.225564
247  230360     에코마케팅      11400      36100  216.666667
25   138080     오이솔루션      13295      41605  212.937194
211  190510       나무가       9525      25000  162.467192
19   051160     지어소프트       3225       7800  141.860465
..      ...       ...        ...        ...         ...
251  141020       포티스       2395        995  -58.455115
28   260970      에스앤디     361300     117500  -67.478550
160  103230    에스앤더블류      14550       3950  -72.852234
181  950160    코오롱티슈진      43150       8010  -81.436848
184  121060     유니포인트      16950       1740  -89.734513

[300 rows x 5 columns]

Absolute momentum (2018-12-28 00:00:00 ~ 2019-06-28 00:00:00) : 11.10% 



In [77]:
# 강환국이란 사람에 따르면 게리 안토나치가 주로 12개월 듀얼 모멘텀 전략을 사용했지만, 21세기 한국 시장에서는 3개월 전략이 훨씬 수익률이 좋았다고 함

# 따라서 한국 자산만으로 운영하는 전략에는 3개월 듀얼 모멘텀을 적용하고, 한국 자산과 해외 자산을 혼합하는 경우에는 12개월 듀얼 모멘텀을 적용하는 한국형 전략도 고려해볼 만함

# 또한 90일 미국 국채를 직접 매수하는 대신 TLT 같은 ETF에 투자할 경우, 미국 국채 수익률을 추종하면서 환 헤지까지 할 수 있는 한국 투자자만의 장점이 있기도 함

# 3개월 전략을 사용할 경우 (같은 3개월 조건에서는 종목수가 적을수록 수익률이 증가함. 여기서는 10개로 테스트)
df_relative = get_rltv_momentum('2019-05-15', '2019-08-15', 10)
print(df_relative)
df_absolute = get_abs_momentum(df_relative,'2019-08-15', '2019-11-15')
print(df_absolute)


code company  old_price  new_price     returns
0  023770   플레이위드       3197      16262  408.664373
1  002070   남영비비안        654       2270  247.094801
2  207760   미스터블루       2820       8420  198.581560
3  258610  이더블유케이       3920      10850  176.785714
4  074610   이엔플러스       3680       9820  166.847826
5  005360     모나미       2670       6870  157.303371
6  096040     이트론        165        402  143.636364
7  032500  케이엠더블유      24250      58100  139.587629
8  179720    옐로페이        169        400  136.686391
9  088290   이원컴포텍       1405       3260  132.028470

Relative momentum (2019-05-14 00:00:00 ~ 2019-08-14 00:00:00) : 190.72% 

     code company  old_price  new_price     returns
0  023770   플레이위드       3197      16262  408.664373
1  002070   남영비비안        654       2270  247.094801
2  207760   미스터블루       2820       8420  198.581560
3  258610  이더블유케이       3920      10850  176.785714
4  074610   이엔플러스       3680       9820  166.847826
5  005360     모나미       2670       6870  157.3